In [3]:
import sqlite3
import random
from datetime import datetime

conn = sqlite3.connect('Employees_performance.db')
c = conn.cursor()

# Drop tables in dependency order
c.execute("DROP TABLE IF EXISTS Reviews")
c.execute("DROP TABLE IF EXISTS Tasks")
c.execute("DROP TABLE IF EXISTS Projects")
c.execute("DROP TABLE IF EXISTS Employees")
conn.commit()

# Employees table
c.execute("""
CREATE TABLE Employees (
    emp_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT,
    years_at_company INTEGER CHECK(years_at_company >= 0),
    salary REAL CHECK(salary >= 0)
)
""")

# Projects table
c.execute("""
CREATE TABLE Projects (
    project_id INTEGER PRIMARY KEY AUTOINCREMENT,
    project_name TEXT NOT NULL UNIQUE,
    start_date DATE,
    end_date DATE
)
""")

# Tasks table (each assigned to one employee and part of one project)
c.execute("""
CREATE TABLE Tasks (
    task_id INTEGER PRIMARY KEY AUTOINCREMENT,
    project_id INTEGER,
    emp_id INTEGER,
    task_name TEXT NOT NULL,
    due_date DATE,
    status TEXT CHECK(status IN ('Not Started', 'In Progress', 'Complete')),
    FOREIGN KEY(project_id) REFERENCES Projects(project_id),
    FOREIGN KEY(emp_id) REFERENCES Employees(emp_id)
)
""")

# Reviews table
c.execute("""
CREATE TABLE Reviews (
    emp_id INTEGER,
    review_year INTEGER,
    performance_rating TEXT CHECK(performance_rating IN ('Poor', 'Fair', 'Good', 'Very Good', 'Excellent')),
    PRIMARY KEY (emp_id, review_year),
    FOREIGN KEY(emp_id) REFERENCES Employees(emp_id)
)
""")
conn.commit()

# Insert projects
project_names = [
    'CRM Upgrade', 'Website Redesign', 'Cloud Migration',
    'Mobile App Launch', 'Security Audit', 'Data Warehouse Integration', 'AI Chatbot Launch'
]
projects = []
current_year = datetime.now().year
for name in project_names:
    start = f"{current_year - random.randint(0, 3)}-01-01"
    end = f"{current_year - random.randint(0, 1)}-12-31"
    projects.append((name, start, end))
c.executemany("INSERT INTO Projects (project_name, start_date, end_date) VALUES (?, ?, ?)", projects)
conn.commit()

# Insert employees
first_names = [
    'Olivia', 'Liam', 'Emma', 'Noah', 'Ava', 'Sophia', 'James', 'Isabella',
    'Benjamin', 'Charlotte', 'Mia', 'Elijah', 'Amelia', 'Lucas', 'Harper',
    'Mason', 'Evelyn', 'Logan', 'Abigail', 'Jacob'
]
last_names = [
    'Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia', 'Miller',
    'Davis', 'Rodriguez', 'Martinez', 'Hernandez', 'Lopez', 'Gonzalez',
    'Wilson', 'Anderson', 'Thomas', 'Taylor', 'Moore', 'Jackson', 'Martin'
]
employee_data = []
for i in range(1500):
    first = random.choice(first_names)
    last = random.choice(last_names)
    email = f"{first.lower()}.{last.lower()}@company.com"
    if i % 100 == 0 and i != 0:
        email = "duplicate.email@company.com"
    years = random.randint(0, 40)
    salary = round(random.uniform(30000, 150000), 2) if random.random() > 0.05 else None
    employee_data.append((first, last, email, years, salary))
c.executemany("""
INSERT INTO Employees (first_name, last_name, email, years_at_company, salary)
VALUES (?, ?, ?, ?, ?)""", employee_data)
conn.commit()

# Insert tasks: each project gets a random number of tasks, each assigned to an employee
status_options = ['Not Started', 'In Progress', 'Complete']
task_titles = [
    'Design', 'Implementation', 'Testing', 'Documentation', 'Deployment',
    'User Training', 'Integration', 'QA Review', 'Performance Tuning'
]
task_data = []
for proj_id in range(1, len(project_names) + 1):
    num_tasks = random.randint(10, 30)
    for _ in range(num_tasks):
        emp_id = random.randint(1, 1500)
        task_name = f"{random.choice(task_titles)} #{random.randint(1, 100)}"
        due = f"{current_year}-{random.randint(1,12):02d}-{random.randint(1,28):02d}"
        status = random.choice(status_options)
        task_data.append((proj_id, emp_id, task_name, due, status))
c.executemany("""
INSERT INTO Tasks (project_id, emp_id, task_name, due_date, status)
VALUES (?, ?, ?, ?, ?)""", task_data)
conn.commit()

# Reviews (like before)
performance_ratings = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']
review_data = []
for emp_id in range(1, 1501):
    review_year = random.randint(current_year - 5, current_year)
    rating = random.choice(performance_ratings) if random.random() > 0.10 else None
    review_data.append((emp_id, review_year, rating))
c.executemany("""
INSERT INTO Reviews (emp_id, review_year, performance_rating)
VALUES (?, ?, ?)""", review_data)
conn.commit()

# Audit output
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cur.fetchall()]
for table in tables:
    print(f"\nTable: {table}")
    cur.execute(f"PRAGMA table_info({table});")
    columns_info = cur.fetchall()
    columns = [info[1] for info in columns_info]
    print("Columns:", columns)
    cur.execute(f"SELECT COUNT(*) FROM {table};")
    total_rows = cur.fetchone()[0]
    print("Total rows:", total_rows)
    for col in columns:
        cur.execute(f"SELECT COUNT(*) FROM {table} WHERE {col} IS NULL;")
        null_count = cur.fetchone()[0]
        print(f"Missing values in column '{col}':", null_count)
    cur.execute(f"SELECT * FROM {table} LIMIT 5;")
    sample_rows = cur.fetchall()
    print("Sample rows:")
    for row in sample_rows:
        row_dict = dict(zip(columns, row))
        print(row_dict)
conn.close()



Table: Employees
Columns: ['emp_id', 'first_name', 'last_name', 'email', 'years_at_company', 'salary']
Total rows: 1500
Missing values in column 'emp_id': 0
Missing values in column 'first_name': 0
Missing values in column 'last_name': 0
Missing values in column 'email': 0
Missing values in column 'years_at_company': 0
Missing values in column 'salary': 86
Sample rows:
{'emp_id': 1, 'first_name': 'Noah', 'last_name': 'Gonzalez', 'email': 'noah.gonzalez@company.com', 'years_at_company': 24, 'salary': 85906.25}
{'emp_id': 2, 'first_name': 'James', 'last_name': 'Miller', 'email': 'james.miller@company.com', 'years_at_company': 37, 'salary': 124350.53}
{'emp_id': 3, 'first_name': 'Mia', 'last_name': 'Taylor', 'email': 'mia.taylor@company.com', 'years_at_company': 6, 'salary': 93236.94}
{'emp_id': 4, 'first_name': 'Sophia', 'last_name': 'Martin', 'email': 'sophia.martin@company.com', 'years_at_company': 21, 'salary': 81373.87}
{'emp_id': 5, 'first_name': 'Amelia', 'last_name': 'Martin', 'e